In [ ]:
import os
os.makedirs('/tmp/practical', exist_ok=True)


In [2]:
!powershell -Command "Get-Help New-Item -Detailed"
!powershell -Command "New-Item -ItemType File -Path linux -Force"



NAME
    New-Item
    
SYNTAX
    New-Item [-Path] <string[]> [-ItemType <string>] [-Value <Object>] [-Force] [-Credential <pscredential>] [-WhatIf] 
    [-Confirm] [-UseTransaction]  [<CommonParameters>]
    
    New-Item [[-Path] <string[]>] -Name <string> [-ItemType <string>] [-Value <Object>] [-Force] [-Credential 
    <pscredential>] [-WhatIf] [-Confirm] [-UseTransaction]  [<CommonParameters>]
    
    
PARAMETERS
    -Confirm
    
    -Credential <pscredential>
    
    -Force
    
    -ItemType <string>
    
    -Name <string>
    
    -Path <string[]>
    
    -UseTransaction
    
    -Value <Object>
    
    -WhatIf
    
    <CommonParameters>
        This cmdlet supports the common parameters: Verbose, Debug,
        ErrorAction, ErrorVariable, WarningAction, WarningVariable,
        OutBuffer, PipelineVariable, and OutVariable. For more information, see 
        about_CommonParameters (https:/go.microsoft.com/fwlink/?LinkID=113216). 
    

ALIASES
    ni
    

REMARKS
    G

In [3]:
!powershell -Command "Set-Content -Path linux -Value '#!/bin/sh'"
!powershell -Command "Add-Content -Path linux -Value 'curl --head --silent https://missing.csail.mit.edu'"


In [11]:
with open('linux', 'w', newline='\n') as f:
    f.write('#!/bin/sh\n')
    f.write('curl --head --silent https://missing.csail.mit.edu\n')


In [12]:
import os
import stat
import subprocess

try:
    subprocess.run(['./linux'], check=True)
except Exception as e:
    print('Execution failed:', e)

mode = os.stat('linux').st_mode
print('Mode bits:', oct(mode & 0o777))
print('Symbolic mode:', stat.filemode(mode))
print('No execute bit set, so direct ./linux execution fails.')


Execution failed: [WinError 193] %1 is not a valid Win32 application
Mode bits: 0o666
Symbolic mode: -rw-rw-rw-
No execute bit set, so direct ./linux execution fails.


In [14]:
import os
import shutil
import subprocess

sh_cmd = shutil.which('sh')
if not sh_cmd:
    git_sh = r'C:\\Program Files\\Git\\bin\\sh.exe'
    if os.path.exists(git_sh):
        sh_cmd = git_sh

if sh_cmd:
    result = subprocess.run([sh_cmd, 'linux'], capture_output=True, text=True)
    print('Exit code:', result.returncode)
    if result.stdout:
        print(result.stdout)
    if result.stderr:
        print(result.stderr)
else:
    print("'sh' is not installed on this Windows environment, so 'sh linux' cannot run here.")
print('sh linux works because sh reads the script as input; ./linux requires execute permission on the file.')


Exit code: 0
HTTP/1.1 200 OK
Connection: keep-alive
Content-Length: 11116
Server: GitHub.com
Content-Type: text/html; charset=utf-8
Last-Modified: Tue, 24 Feb 2026 15:44:44 GMT
Access-Control-Allow-Origin: *
ETag: "699dc76c-2b6c"
expires: Fri, 27 Feb 2026 10:26:51 GMT
Cache-Control: max-age=600
x-proxy-cache: MISS
X-GitHub-Request-Id: 3E2A:825FB:E1A4:EFA9:69A16F11
Accept-Ranges: bytes
Age: 0
Date: Fri, 27 Feb 2026 10:39:02 GMT
Via: 1.1 varnish
X-Served-By: cache-bkk2310022-BKK
X-Cache: HIT
X-Cache-Hits: 0
X-Timer: S1772188742.097725,VS0,VE285
Vary: Accept-Encoding
X-Fastly-Request-ID: 1df30decdbf996f14f76659c29650ea506b9642c


sh linux works because sh reads the script as input; ./linux requires execute permission on the file.


In [15]:
import os
import shutil
import stat
import subprocess

print('Trying to look up chmod with man:')
man_cmd = shutil.which('man')
if man_cmd:
    subprocess.run([man_cmd, 'chmod'])
else:
    print("'man' is not available in this environment.")

chmod_cmd = shutil.which('chmod')
if chmod_cmd:
    subprocess.run([chmod_cmd, '+x', 'linux'], check=True)
else:
    os.chmod('linux', os.stat('linux').st_mode | stat.S_IXUSR)
    print("'chmod' command not found; used Python os.chmod to add user execute bit.")

print('New mode:', oct(os.stat('linux').st_mode & 0o777), stat.filemode(os.stat('linux').st_mode))

print('Trying ./linux:')
try:
    subprocess.run(['./linux'], check=True)
except Exception as e:
    print('Direct execution still failed here:', e)

print('How shell knows to use sh: the shebang line #!/bin/sh at the top of the file tells the kernel which interpreter to run.')


Trying to look up chmod with man:
'man' is not available in this environment.
'chmod' command not found; used Python os.chmod to add user execute bit.
New mode: 0o666 -rw-rw-rw-
Trying ./linux:
Direct execution still failed here: [WinError 193] %1 is not a valid Win32 application
How shell knows to use sh: the shebang line #!/bin/sh at the top of the file tells the kernel which interpreter to run.


In [16]:
import os
import shutil
import subprocess

sh_cmd = shutil.which('sh')
if not sh_cmd:
    git_sh = r'C:\\Program Files\\Git\\bin\\sh.exe'
    if os.path.exists(git_sh):
        sh_cmd = git_sh

if sh_cmd:
    cmd = "sh linux | grep -i '^Last-Modified:' > ~/last-modified.txt"
    subprocess.run([sh_cmd, '-lc', cmd], check=True)
    out_file = os.path.expanduser('~/last-modified.txt')
    print('Wrote:', out_file)
    with open(out_file, 'r', encoding='utf-8') as f:
        print(f.read())
else:
    print("'sh' is not available, so the pipe/redirection command cannot run in this environment.")


Wrote: C:\Users\HP OMEN/last-modified.txt
Last-Modified: Tue, 24 Feb 2026 15:44:44 GMT



In [21]:
import subprocess
subprocess.run("cat /sys/class/power_supply/BAT0/capacity || cat /sys/class/thermal/thermal_zone0/temp", shell=True)


CompletedProcess(args='cat /sys/class/power_supply/BAT0/capacity || cat /sys/class/thermal/thermal_zone0/temp', returncode=1)